In [ ]:
# !pip install Bvh

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bvh import Bvh
from tqdm import tqdm
from matplotlib.animation import FuncAnimation

In [ ]:
# data from mocap_xia in gitrepo deep-motion-editing

DATA_DIR = "sample_data/"
CACHE_DIR = "cache_files/"
OUTPUT_DIR = "output/"
data_files = os.listdir(DATA_DIR)

sample_file = data_files[0]

In [ ]:
with open(DATA_DIR + sample_file) as f:
    bvh_data = Bvh(f.read())

In [ ]:
print("len each frame: ", len(bvh_data.frames[0]))

for name in bvh_data.get_joints_names():
    print(name, "->", bvh_data.get_joint_channels_index(name))

# Get the motion data (frames)

In [ ]:
bvh_data.frames[0][84:87]

In [ ]:
print(bvh_data.frame_joint_channels(0, 'Hips', ['Xposition', 'Yposition', 'Zposition']))
print(bvh_data.frame_joint_channels(0, 'RightHand', ['Xrotation', 'Yrotation', 'Zrotation']))
print(bvh_data.joint_offset("Hips"))
print(bvh_data.joint_offset("RightHand"))

In [ ]:
def get_df(file):

    if not os.path.exists(CACHE_DIR):
        os.makedirs(CACHE_DIR)

    file_path = DATA_DIR + file
    if not os.path.exists("%s/%s_worldpos.csv" % (CACHE_DIR, file[:-4])):
        os.system("cp %s %s" % (file_path, CACHE_DIR))
        os.system("bvh-converter %s/%s" % (CACHE_DIR, file))
        os.system("rm %s/%s" % (CACHE_DIR, file))
    return pd.read_csv("%s/%s_worldpos.csv" % (CACHE_DIR, file[:-4]))

In [ ]:
df = get_df(sample_file)
df

In [ ]:
def plot_frame_mocap(df, bvh_data, framIdx):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection='3d')

    ax.set_xlim(-10, 25)
    ax.set_ylim(-10, 25)
    ax.set_zlim(-10, 25)

    X, Y, Z= [], [], []
    for name in bvh_data.get_joints_names():
        parent_idx = bvh_data.joint_parent_index(name)
        if parent_idx >= 0:
            parent_name = bvh_data.get_joints_names()[parent_idx]
        else:
            parent_name = None
            continue
        
        # print(name, parent_name)
        x1 = df.iloc[framIdx][name + ".X"]
        y1 = df.iloc[framIdx][name + ".Y"]
        z1 = df.iloc[framIdx][name + ".Z"]

        x2 = df.iloc[framIdx][parent_name + ".X"]
        y2 = df.iloc[framIdx][parent_name + ".Y"]
        z2 = df.iloc[framIdx][parent_name + ".Z"]

        # TODO be why?
        ax.plot([x1, x2], [z1, z2] ,[y1, y2],  "bo-")

    plt.show()

plot_frame_mocap(df, bvh_data, 0)

In [ ]:
content_full_names = [
    "walk_s1", "walk_s2",
    "walk_lturn_s1", "walk_rturn_s1", "walk_lturn_l1", "walk_lturn_l2",
    "walk_l1", "walk_l2",
    "walk_rturn_s2", "walk_lturn_s2", "walk_rturn_l1", "walk_lturn_l3",
    "run", "run_lturn", "run_rturn",
    "jump_1", "jump_2",
    "punch_r", "punch_l", "punch_qr", "punch_ql",
    "kick_l", "kick_r",
    "trans_jump2walk", "trans_walk2jump", "trans_punch2kick", "trans_walk2punch", "trans_run2jump"]


# number of classes e.g. angry_12_000.bvh means angry class 12
len(content_full_names), content_full_names[12-1], content_full_names[17-1], content_full_names[20-1]

In [ ]:
('pelvis', 'left_hip'), ('pelvis', 'right_hip'),
('left_hip', 'left_knee'), ('right_hip', 'right_knee'),
('left_knee', 'left_ankle'), ('right_knee', 'right_ankle'),
('left_ankle', 'left_foot'), ('right_ankle', 'right_foot'),
('pelvis', 'spine1'),
('spine1', 'spine2'),
('spine2', 'spine3'),
('spine3', 'neck'),
('left_collar', 'right_collar'),
('right_collar', 'right_wrist'),
('right_wrist', 'right_index2'),
('left_collar', 'left_index3')

joint_mapping = {
    'left_foot': 'LeftToeBase',
    'left_ankle': 'LeftFoot',
    'left_knee': 'LeftLeg',
    'left_hip': 'LeftUpLeg',

    'right_foot': 'RightToeBase',
    'right_ankle': 'RightFoot',
    'right_knee': 'RightLeg',
    'right_hip': 'RightUpLeg',

    'pelvis': 'Hips',


    'spine1': 'LowerBack',
    'spine2': 'Spine',
    'spine3': 'Spine1',
    'neck': 'Head',

    'left_collar': 'LeftArm',
    'right_collar': 'RightArm',

    'right_wrist': 'RightHand',
    'left_index3': 'LeftHandIndex1End',
    'right_index2': 'RightHandIndex1End',
}



In [ ]:
# joint in amass dataset
joints = ['right_ankle', 'left_hip', 'right_hip', 'spine1', 'left_knee', 'right_knee', 'spine2', 'left_ankle', 'pelvis', 'spine3', 'left_foot', 'right_foot', 'neck', 'left_collar', 'right_collar', 'right_wrist', 'left_index3', 'right_index2']
print(len(joints))
print(joints)
joints_to_index = {j:i for i, j in enumerate(joints)}
print(joints_to_index)

In [ ]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

VISUALIZE_OUTPUT_DIR = OUTPUT_DIR+"/visualize"
if not os.path.exists(VISUALIZE_OUTPUT_DIR):
    os.makedirs(VISUALIZE_OUTPUT_DIR)



for file in tqdm(sorted(data_files)):

  df = get_df(file)

  with open(DATA_DIR + file) as f:
    bvh_data = Bvh(f.read())



  # Set up figure and 3D axes
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(projection='3d')



  # Function to update the plot for each frame
  def update(frame_idx):
      ax.cla()  # Clear previous frame

      ax.set_title(content_full_names[int(file[-10:-8]) - 1])

      ax.set_xlim(-10, 25)
      ax.set_ylim(-10, 25)
      ax.set_zlim(-10, 25)


      for name in bvh_data.get_joints_names():
          parent_idx = bvh_data.joint_parent_index(name)
          if parent_idx >= 0:
              parent_name = bvh_data.get_joints_names()[parent_idx]
          else:
              parent_name = None
              continue
          
          # Fetch the joint positions for the current frame
          x1 = df.iloc[frame_idx][name + ".X"]
          y1 = df.iloc[frame_idx][name + ".Y"]
          z1 = df.iloc[frame_idx][name + ".Z"]

          x2 = df.iloc[frame_idx][parent_name + ".X"]
          y2 = df.iloc[frame_idx][parent_name + ".Y"]
          z2 = df.iloc[frame_idx][parent_name + ".Z"]

          # Create plot for the connection between joints
          ax.plot([x1, x2], [z1, z2], [y1, y2], "bo-")

          # Collect joint positions for the scatter plot

      # Optionally add scatter plot for joint positions
      
      plt.draw()

  # Create the animation
  fps = int(1/bvh_data.frame_time)
  ani = FuncAnimation(fig, update, frames=df.shape[0], interval=1000/fps, repeat=False)

  # Save the animation as a GIF
  # ani.save('skeleton_animation.gif', writer='pillow', fps=fps, dpi=80)
  ani.save('%s/mocap_sample_data_%s.mp4' % (VISUALIZE_OUTPUT_DIR, file[:file.find(".")]), writer='ffmpeg', fps=fps, dpi=80)


  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111, projection='3d')
  data_index=1

  def update(frame_idx):
      ax.cla()

      ax.set_title(content_full_names[int(file[-10:-8]) - 1])

      ax.set_xlim(-10, 25)
      ax.set_ylim(-10, 25)
      ax.set_zlim(-10, 25)


      for v in [
          ('pelvis', 'left_hip'), ('pelvis', 'right_hip'),
          ('left_hip', 'left_knee'), ('right_hip', 'right_knee'),
          ('left_knee', 'left_ankle'), ('right_knee', 'right_ankle'),
          ('left_ankle', 'left_foot'), ('right_ankle', 'right_foot'),
          ('pelvis', 'spine1'),
          ('spine1', 'spine2'),
          ('spine2', 'spine3'),
          ('spine3', 'neck'),
          ('left_collar', 'right_collar'),
          ('right_collar', 'right_wrist'),
          ('right_wrist', 'right_index2'),
          ('left_collar', 'left_index3')
      ]:
          # vec = [joints_to_index[j] for j in v]

          vec = [joint_mapping[j]+".X" for j in v]
          x = (df.iloc[frame_idx][vec].to_numpy())

          vec = [joint_mapping[j]+".Y" for j in v]
          z = (df.iloc[frame_idx][vec].to_numpy())

          vec = [joint_mapping[j]+".Z" for j in v]
          y = (df.iloc[frame_idx][vec].to_numpy())

          ax.scatter(x, y, z, c='b', marker='o')  # Scatter plot for the points
          ax.plot(xs=x, ys=y, zs=z, c='r', linestyle='-', marker='o')


      # Set labels for the axes
      ax.set_xlabel('X Axis')
      ax.set_ylabel('Y Axis')
      ax.set_zlabel('Z Axis')

      # Set the title
      # ax.set_title(data["text_raw_labels"][data_index][frame_idx])

      # Show the plot
      plt.draw()# 


  fps = int(1/bvh_data.frame_time)
  ani = FuncAnimation(fig, update, frames=df.shape[0], interval=1000/fps, repeat=False)

  ani.save('%s/mocap_sample_data_%s_in_amass_format.mp4' % (VISUALIZE_OUTPUT_DIR, file[:file.find(".")]), writer='ffmpeg', fps=fps, dpi=80)

In [ ]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

DATA_OUTPUT_DIR = OUTPUT_DIR+"/data"
if not os.path.exists(DATA_OUTPUT_DIR):
    os.makedirs(DATA_OUTPUT_DIR)

In [ ]:
for file in tqdm(sorted(data_files)):

    df = get_df(file)

    res = np.stack([
        df[[joint_mapping[j]+".X" for j in joints]].to_numpy(),
        df[[joint_mapping[j]+".Z" for j in joints]].to_numpy(),           # HINT: in amass dataset Z and Y are different from here
        df[[joint_mapping[j]+".Y" for j in joints]].to_numpy()], axis=-1)

    np.save(DATA_OUTPUT_DIR + "/" + file[:-4] + ".npy", res)

In [ ]:
np.load(DATA_OUTPUT_DIR + "/" + data_files[0][:-4] + ".npy").shape